In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.utils import get_target_box_number_density
import unyt as u
import hoomd

/Users/stephaniemccallum/miniforge3/envs/flowermd/lib/python3.11/site-packages/mdtraj/formats/__init__.py:13: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from mdtraj.formats.trr import TRRTrajectoryFile



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



## System Initialization

In [2]:
from flowermd.base import Pack,Lattice, Simulation
from flowermd.library import EllipsoidChain

ellipsoid_chain = EllipsoidChain(lengths=2,num_mols=1,lpar=1.0,bead_mass=1.0)
system = Pack(molecules=ellipsoid_chain, density=0.05*u.Unit("nm**-3"), packing_expand_factor=6,edge=2,overlap=1,fix_orientation=True)

## Forcefield definition

In [3]:
from flowermd.library import EllipsoidForcefield

ff = EllipsoidForcefield(epsilon=1.0,lpar=1.0,lperp=0.5,r_cut=2.0,bond_k=100,bond_r0=0,angle_k=30,angle_theta0=1.9)

## Rigid Body Constraint

In [4]:
from flowermd.utils.constraints import create_rigid_ellipsoid_chain

rigid_frame, rigid = create_rigid_ellipsoid_chain(system.hoomd_snapshot)

## Simulation

In [5]:
gsd_path=('ellipsoid-chain-2mer.gsd')
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.001,
    gsd_write_freq=int(1e4),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name='log.txt')

target_box = get_target_box_number_density(density=0.01*u.Unit("nm**-3"),n_beads=2)
ellipsoid_sim.run_update_volume(final_box_lengths=target_box, kT=7.0, n_steps=1e5,tau_kt=100*ellipsoid_sim.dt,period=10,thermalize_particles=True)
print("shrink finished")
ellipsoid_sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10*ellipsoid_sim.dt)
#ellipsoid_sim.save_restart_gsd("restart.gsd")
ellipsoid_sim.flush_writers()
#ellipsoid_sim.save_simulation("sim.pickle")

*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 10000 of 100000; TPS: 54734.84; ETA: 0.0 minutes
Step 20000 of 100000; TPS: 94268.48; ETA: 0.0 minutes
Step 30000 of 100000; TPS: 126955.11; ETA: 0.0 minutes
Step 40000 of 100000; TPS: 153201.53; ETA: 0.0 minutes
Step 50000 of 100000; TPS: 177389.26; ETA: 0.0 minutes
Step 60000 of 100000; TPS: 198780.15; ETA: 0.0 minutes
Step 70000 of 100000; TPS: 216717.6; ETA: 0.0 minutes
Step 80000 of 100000; TPS: 233536.41; ETA: 0.0 minutes
Step 90000 of 100000; TPS: 248460.24; ETA: 0.0 minutes
Step 100000 of 100000; TPS: 261405.1; ETA: 0.0 minutes
shrink finished
Step 9999 of 100000; TPS: 517385.9; ETA: 0.0 minutes
Step 19999 of 100000; TPS: 519981.28; ETA: 0.0 minutes
Step 29999 of 100000; TPS: 520436.49; ETA: 0.0 minutes
Step 39999 of 100000; TPS: 520156.57; ETA: 0.0 minutes
Step 49999 of 100000; TPS: 518586.51; ETA: 0.0 minutes
Step 59999 of 100000; TPS: 518032.14; ETA: 0.0 minutes
Step 69999 of 100000; TPS: 518365.2; ETA: 0.0 minutes
S

## Function to redefine rigid constraint for restarting simulation

In [7]:
def restart_rigid_ellipsoid():
    local_coords = [(0.0, 0.0, 0.0), (1.049999999999999, 0.0, 0.0), (1.0, 0.0, 0.0), (-1.0000000000000009, 0.0, 0.0)]
    rigid_constrain = hoomd.md.constrain.Rigid()
    rigid_constrain.body["R"] = {
        "constituent_types": ["X", "A", "T", "T"],
        "positions": local_coords,
        "orientations": [[1, 0, 0, 0]] * len(local_coords),
    }
    return rigid_constrain

In [6]:
ff = EllipsoidForcefield(epsilon=1.0,lpar=1.0,lperp=0.5,r_cut=2.0,bond_k=100,bond_r0=0,angle_k=30,angle_theta0=1.9)
rigid = restart_rigid_ellipsoid_sim()
gsd_path = "sim_restart.gsd"

sim = Simulation(
    initial_state="restart.gsd",
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    gsd_write_freq=int(1e3),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name='log2.txt',
)
sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10*sim.dt)
sim.flush_writers()

NameError: name 'restart_rigid_ellipsoid_sim' is not defined

## Adding ellispoid information to GSD 

In [8]:
from cmeutils.gsd_utils import ellipsoid_gsd 
ellipsoid_gsd(gsd_file=gsd_path,new_file="ovito-ellipsoid.gsd",lpar=1.0,lperp=0.5)